In [1]:
#Import need libraries 

import numpy as np
import matplotlib.pyplot as plt

# Static Rocket Simulation

We want to simulate a rocket without any moving parts - first think about the different frames of referecene

A real rocket will have sensors telling it it's position, velocity, and acceleration, relative to itself, not relative to the ground - we want to work in two different coordinate framses

**Inertial Frame** - frame relative to the ground - origin is at the point of launch

**Body Frame** - frame relative to the rocket - origin is at the center of mass of the rocket

We look at the forces acting on the rocket using the inertial frame (it is easier to see what those are this way) and then translate them to the body frame to study the evolution of the rocket 

#### How to do this ?

We coudl try to use euler angles to describe the rotations, but this is messy and introduces gimble lock, where we can have two axis coinciding over each other thus making us loose some degrees of freedom. 

We will want to use quaternions instead

## What are quaternions ?

Quaterniosn are a mathematical tool invented by William Rowan Hamilton, it is a system qith 4 complex units and the same way that a multiplication by i is a 90º rotation, we can establish a relationship between these and 3D rotations. 

We can define a quaternion using a scalar and a 3D vector:

$ q = (a, \underline b) $ or using another notation $q = a + \underline b$ or $q = a + b_1 i + b_2 j + b_3 k$, where i,j, k are 3 independent complex numbers with the property : $ij = -ji = k, jk= -kj = i , ki = -ik = j $ and $i^2 = j^2 = k^2 = -1$

In this theory, vectors can be seen as pure quaternions

### Basic Properties of Quaternions 

**Addition** - $q_1 + q_2 = (a_1 + a_2, \underline b_1 + \underline b_2 )$

**Multiplication** - $q_1 q_2 = (a_1a_2 - \underline b_1 \cdot \underline b_2 , a_1 \underline b_2 + a_2 \underline b_1 + \underline b_1 \times \underline b_2)$

Now if we want to use quaternions we must define functions for these properties.

In [ ]:
def quaternion_multiplication(q1, q2):
    
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    return np.array([
        w1*w2 - x1*x2 - y1*y2 - z1*z2,
        w1*x2 + x1*w2 + y1*z2 - z1*y2,
        w1*y2 - x1*z2 + y1*w2 + z1*x2,
        w1*z2 + x1*y2 - y1*x2 + z1*w2
    ])

def quaternion_sum(q1, q2):

    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    
    return np.array([
        w1 + w2,
        -(x1 + x2),
        -(y1 + y2),
        -(z1 + z2)
    ]) 

## Quaternions and Rotations

Now we need to understand how quaternions can be translated into 3D rotations.

First, we can describe a vector in 3D using a quaternion by using pure quaternions $(0, \underline v)$. Using an unit quaternion we have a rotation operator $L(q) = q \underline v q^*$, where q is a unit quaternion described as $q = q_0 + \underline q = cos(\frac{\theta}{2} ) + sin(\frac{\theta}{2} )\underline \mu$

The equivalence between a quaternion ($q = (w,x,y,z)$) and a rotation matrix is given by such that $L(q) = R q$:

$$ R =  \begin{pmatrix}

    1 - 2 (y^2 +z^2) & 2(xy-zw) & 2(xz + yw) \\
    2(xy+zw) & 1 - 2 (x^2 +z^2) &  2(yz + xw) \\
    2(xz - yw) & 2(yz + xw) & 1 - 2 (x^2 +y^2)
\end{pmatrix} $$


This can be also written for a given axis with $q = cos(\frac{\theta}{2} ) + sin(\frac{\theta}{2} )\underline n$, where $\underline n$ is the vector director of the rotation axis, in such a way that :

$a = cos(\frac{\theta}{2} )$

$b = - n_x sin(\frac{\theta}{2} )$

$c = - n_y sin(\frac{\theta}{2} )$

$d = - n_z sin(\frac{\theta}{2} )$


$$ R =  \begin{pmatrix}

    a^2 + b^2 - c^2 - d^2 & 2(bc-ad) & 2(bd + ac) \\
    2(bc+ad) & a^2 + c^2 - b^2 - d^2  &  2(cd - ab) \\
    2(bd - ac) & 2(cd + ab) & a^2 + d^2 - b^2 - c^2 
\end{pmatrix} $$

In [15]:
def quaternion_to_rotation_matrix(q):
    w, x, y, z = q
    return np.array([
        [1-2*(y**2+z**2), 2*(x*y-w*z), 2*(x*z+w*y)],
        [2*(x*y+w*z), 1-2*(x**2+z**2), 2*(y*z-w*x)],
        [2*(x*z-w*y), 2*(y*z+w*x), 1-2*(x**2+y**2)]
    ])
    
def normalize_quaternion(q):
    return q / np.linalg.norm(q)

def axis_angle_rotation(axis, theta):
    axis = axis / np.linalg.norm(axis)
    a = np.cos(theta / 2)
    b, c, d = -axis * np.sin(theta / 2)
    return np.array([
        [a*a + b*b - c*c - d*d, 2*(b*c - a*d), 2*(b*d + a*c)],
        [2*(b*c + a*d), a*a + c*c - b*b - d*d, 2*(c*d - a*b)],
        [2*(b*d - a*c), 2*(c*d + a*b), a*a + d*d - b*b - c*c]
    ])


this is equivalent to have a rotation matrix around the 3 axis of the the rocket 

yaw 
pitch 
roll 
<img src="Rocket_diagram.webp" alt="Rocket Diagram" width="150"/>


Which means that we can also find a rotation matrix for angles around this body, using the roll ($\alpha$), yaw ($\beta$) and pitch ($\theta$) angles.

Equivalence between Euler Angles and Quaternions:

$$\alpha = tan^{-1} \left(\frac{2(q_2q_1 + q_3q_4)}{1 -2 (q_2^2 + q_3^2)}\right);

\beta = sin^{-1} (2(q_1q_3 - q_2q_4));  

\theta = tan^{-1} \left(\frac{2(q_1q_4 - q_3q_2)}{1 -2 (q_3^2 + q_4^2)}\right)$$

# Forces on the rocket and how to simulate them 

### What are the forces acting on the rocket ?

- Gravity 
- Drag 
- Wind
- Initial force from the engine (determined by a trust profile - start by simulating it before implementing the real thing)

Don't forget you have the bottom fins at a fixed angle (I think for now I will ignore this but it could be relevant) as well as the movable fins at the top

So what are the equations we must obey ? - We need to consider the force on the rocket and the updates on the Inertial Frame, the angular velocities in the body frame as well as the evolution of the quaternion representing the rotation between the body frame and the 

Drag Equation : $$ F_{Drag} = \frac{1}{2} \rho v^2 A C_D sin(\theta)$$

$\rho$ - Air density at sea level , $v$ - velocity modulus  , $C_D$ - Drag Coefficient , $A$ - Surface Area exposed 

so we have 3 types of kinematics :

- Translational Motion : $F_total = m \underline a$ , $\underline x = \underline x_0 + \underline v t + \frac{1}{2} a t^2$

- Rotational Motion: Euler equation $\underline G = \dot{L} + \underline \omega \times \underline L $, where L is the angular momentum in the Body frame ($\underline L = I \underline \omega$) and I is the moment of inertia tensor in the body frame, which means is a diagonal tensor, giving $\underline G = I \dot{\omega} + \underline \omega \times I \underline \omega $

- Quaternion motion : 



In [16]:
# set constants 

g = 9.81  # acceleration due to gravity in m/s^2
rho =  1.225  # density of air at seas level in kg/m^3

In [ ]:
def thrust_profile(t):
    if t < 3:
        return 15000
    elif t < 4:
        return 15000 * (4 - t)
    else:
        return 0

# Sources 

https://math.unm.edu/~vageli/courses/Ma375/literature/rrr.pdf 

https://leedsrocketry.co.uk/wp-content/uploads/2024/08/Canard-Control-Development.pdf

https://leedsrocketry.co.uk/wp-content/uploads/2024/08/Development-of-a-Canard-Controlled-Active-Stability-System-for-a-Sounding-Rocket.pdf

https://eater.net/quaternions

https://graphics.stanford.edu/courses/cs348a-17-winter/Papers/quaternion.pdf 

https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0276302 

